In [146]:
import numpy as np
import pandas as pd
%matplotlib inline
import summarytools as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

In [147]:
#Importing the data
HIV_df=pd.read_csv(r'C:\Users\rkeoye\Documents\AUDIT_2024\DATA_SCIENCE\PHASE  5\dreams_raw_dataset.csv')
HIV_df.head()#display the first 5 rows of the dataset

,date_of_birth,date_of_enrollment,county,head_of_household,head_of_household_other,age_of_household_head,father_alive,mother_alive,are_both_parents_alive,is_one_parent_alive,...,intervention_name,intervention_date,result,bio_medical,social_protection,behavioral,post_gbv_care,other_interventions,exit_age,exit_reason_other
0,9/16/2004,2/22/2020,Nairobi,Mother,NaN,58.0,Yes,Yes,Yes,No,...,HTS - HTS (Client),8/8/2020,Negative,1,1,1,0,0,NaN,NaN
1,8/2/2004,12/18/2019,Nairobi,Father,NaN,43.0,Yes,Yes,Yes,No,...,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,NaN,NaN
2,10/20/2005,3/7/2020,Nairobi,Mother,NaN,41.0,No,Yes,No,Yes,...,HTS - HTS (Client),8/12/2020,Negative,1,1,0,0,0,NaN,NaN
3,1/18/2006,3/3/2020,Nairobi,Mother,NaN,45.0,No,Yes,No,Yes,...,HTS - HTS (Client),8/12/2020,Negative,1,1,1,0,0,NaN,NaN
4,3/22/2004,12/18/2019,Nairobi,Father,NaN,42.0,Yes,Yes,Yes,No,...,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,NaN,NaN


In [148]:
HIV_df.info()#Getting more information about the dataset
'''The dataset contains 455,807 rows and42 fields Made of the following data types: Float (8 columns, object (29) and int(5))'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455807 entries, 0 to 455806
Data columns (total 42 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   date_of_birth                      455807 non-null  object 
 1   date_of_enrollment                 455806 non-null  object 
 2   county                             446551 non-null  object 
 3   head_of_household                  446192 non-null  object 
 4   head_of_household_other            5960 non-null    object 
 5   age_of_household_head              434789 non-null  float64
 6   father_alive                       442399 non-null  object 
 7   mother_alive                       442820 non-null  object 
 8   are_both_parents_alive             455807 non-null  object 
 9   is_one_parent_alive                455807 non-null  object 
 10  parents_alive_status               390403 non-null  object 
 11  no_of_people_in_household          4486

'The dataset contains 455,807 rows and42 fields Made of the following data types: Float (8 columns, object (29) and int(5))'

In [149]:
st.dfSummary(HIV_df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,date_of_birth[object],1. 1/1/20002. 1/1/20023. 1/1/19984. 1/1/20015. 1/1/19996. 1/1/19967. 1/1/19978. 1/1/19949. 1/1/200310. 1/1/199511. other,"1,011 (0.2%)993 (0.2%)904 (0.2%)883 (0.2%)875 (0.2%)794 (0.2%)756 (0.2%)614 (0.1%)551 (0.1%)509 (0.1%)447,917 (98.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADaElEQVR4nO3cQWoUYRRG0VehiWbSTQhkCbqHLMLFuohehHsIITUwMZNyIIo41lvQnjPK8B9ceDTUl2XbtoHC1d4P4P8hNjJiIyM2MmIjIzYyYiMjNjJiIyM2MlfLsnxcluX93g/h8l3NzIeZebf3Q7h8zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkbmamS8z823vh3D5Fv/FiIozSkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLyQMXgh44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4oGbGRERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjLWVWSsq8g4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4omWVmbmbmeu+HcLHetm17mZk53N3dfToej7d7v4jLtK7r07Isn7dtezkcj8fbh4eHl9Pp9Lr3w7gsz8/P78/n8+3j4+P1zLwcZmZOp9Pr/f39153fxmW6+fmHHwhkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDjM/PgXZ+yFcnj+7Oqzr+nQ+n2/nt09B4G9Z1/VpZt5mfBbOv/frs3DrKjJ+IJARGxmxkREbGbGRERsZsZERGxmxkfkOCvhuJDOJMoEAAAAASUVORK5CYII="">",0(0.0%)
2,date_of_enrollment[object],1. 12/9/20202. 12/2/20203. 12/3/20204. 8/23/20175. 12/7/20206. 8/28/20177. 12/10/20208. 12/4/20209. 12/15/202010. 8/25/201711. other,"1,795 (0.4%)1,786 (0.4%)1,723 (0.4%)1,712 (0.4%)1,707 (0.4%)1,691 (0.4%)1,683 (0.4%)1,673 (0.4%)1,671 (0.4%)1,634 (0.4%)438,732 (96.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADfUlEQVR4nO3cMWobUQBF0S8jTNyMEAItRovwYrMI7UYITREbN0oRpwhRAgH7DijnVCHVFJfR5PNfVtfrdUDhYekH4P8hNjJiIyM2MmIjIzYyYiMjNjJiI7MaYzyNMcb1en1Z+Fm4c+vdbvc8xhir1eqr4PhMD9M0badp2o4xHpd+GO6bbzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiI7Oe5/n8/ue3RZ+Eu2eDQGblv8yi4s1GxrqKjHUVGUcfZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDF7IuBZOxuCFjDcbGYMXMgYvZBx9kBEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXgh41o4GYMXMr7ZyPgZJWNdRca6ioxvNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZFwLJ2NdRcabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxlvNjIGL2QMXsg4+iAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZGwQyFhXkfFmI2NdRca6ioyjDzJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MgYvZFwLJ2PwQsabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxnfbGT8jJKxriJjXUXGNxsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxnrKjKuhZOxriLjzUbG4IWMwQsZRx9kxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGQMXsi4Fk7G4IXMzzebK+F8pLdbv5Tr3W73/L5BgA8xz/P51oBqPU3T9nA4vGw2m9elHo77cblcvhyPx+3pdHocY/wa2xhjbDab1/1+/22Rp+MePd36S+dsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZNZj/LgWsvSDcB/+1tJ6nufz8Xjcjj9cC4F/9T6i+m1A5Vo4n+HmtXCDFzL+gUBGbGTERkZsZMRGRmxkxEZGbGTERuY7CLkQJ3942gEAAAAASUVORK5CYII="">",1(0.0%)
3,county[object],1. Nairobi2. Kisumu3. Homabay4. Migori5. Siaya6. Mombasa7. Kiambu8. nan,"100,411 (22.0%)82,909 (18.2%)79,954 (17.5%)69,846 (15.3%)60,411 (13.3%)31,890 (7.0%)21,130 (4.6%)9,256 (2.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAC4CAYAAAD0WZ4UAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADS0lEQVR4nO3dQWrbaACG4d/FlOnGJgRyGB9iDjuH8G1CiBaT0o1nMQ1kpk2b1NIrqJ5nl5W1ePmNhL54d7lcBhQ+rH0BbIfYyIiNjNjIiI2M2Mjsxhifxhg

In [150]:

# Step 1: Calculate Q1, Q3, and IQR for the "age_of_household_head" column
#Q1 = HIV_df['age_of_household_head'].quantile(0.25)
#Q3 = HIV_df['age_of_household_head'].quantile(0.75)
#IQR = Q3 - Q1

# Step 2: Define the bounds for the outliers
#lower_bound = Q1 - 1.5 * IQR
#upper_bound = Q3 + 1.5 * IQR

# Step 3: Filter the data to remove outliers
#HIV_df= HIV_df[(HIV_df['age_of_household_head'] >= lower_bound) & 
                        #(HIV_df['age_of_household_head'] <= upper_bound)]

# Optional: Check the distribution of the cleaned data
#print(HIV_df['age_of_household_head'].describe())


In [151]:
import pandas as pd
import numpy as np

# Step 1: Identify numerical columns
numerical_cols = HIV_df.select_dtypes(include=[np.number]).columns

# Step 2: Check for outliers using IQR (Interquartile Range) method
outliers_iqr = {}

for col in numerical_cols:
    Q1 = HIV_df[col].quantile(0.25)
    Q3 = HIV_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = HIV_df[(HIV_df[col] < lower_bound) | (HIV_df[col] > upper_bound)]
    outliers_iqr[col] = outliers.shape[0]  # Count of outliers for each column

# Step 3: Check for outliers using Z-score method
from scipy.stats import zscore

outliers_zscore = {}

for col in numerical_cols:
    # Calculate the Z-scores for each value in the column
    z_scores = zscore(HIV_df[col].dropna())
    
    # Identify outliers (Z-score > 3 or < -3)
    outliers = np.where(np.abs(z_scores) > 3)[0]
    outliers_zscore[col] = len(outliers)  # Count of outliers for each column

# Step 4: Combine results
outliers_summary = pd.DataFrame({
    'Outliers (IQR)': outliers_iqr,
    'Outliers (Z-score)': outliers_zscore
})

print(outliers_summary)


                               Outliers (IQR)  Outliers (Z-score)
age_of_household_head                    6374                   9
no_of_people_in_household                8968                 551
no_of_females                            6495                 673
no_of_males                              6147                1215
no_of_adults                            31006                6045
no_of_children                          14999                 410
age_at_first_sexual_encounter            4666                  49
bio_medical                              4276                4276
social_protection                        5422                5422
behavioral                              53648                   0
post_gbv_care                           88147                   0
other_interventions                     27853               27853
exit_age                                 1230                 134


In [152]:
import numpy as np
import pandas as pd

# Define the list of columns to handle (excluding 'age_at_first_sexual_encounter')
columns_to_handle = [
    'age_of_household_head', 
    'no_of_people_in_household', 
    'no_of_females', 
    'no_of_males', 
    'no_of_adults', 
    'no_of_children', 
    'bio_medical', 
    'social_protection', 
    'behavioral', 
    'post_gbv_care', 
    'other_interventions', 
    'exit_age'
]

# Optimized function to handle outliers by replacing them with the median value
def handle_outliers_with_median(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Use boolean masking to replace outliers with the median
        median_value = df[col].median()
        df.loc[(df[col] < lower_bound) | (df[col] > upper_bound), col] = median_value

    return df

# Handle the outliers by replacing them with median values
HIV_df = handle_outliers_with_median(HIV_df, columns_to_handle)

# Optional: Check the cleaned DataFrame
print(HIV_df[columns_to_handle].describe())

       age_of_household_head  no_of_people_in_household  no_of_females  \
count          434789.000000              448607.000000  448475.000000   
mean               42.086042                   5.594300       3.224177   
std                11.992954                   2.120252       1.385272   
min                 8.000000                   0.000000       0.000000   
25%                33.000000                   4.000000       2.000000   
50%                42.000000                   6.000000       3.000000   
75%                50.000000                   7.000000       4.000000   
max                75.000000                  11.000000       7.000000   

         no_of_males   no_of_adults  no_of_children  bio_medical  \
count  448125.000000  448328.000000   448034.000000     455807.0   
mean        2.352337       2.234525        3.053753          1.0   
std         1.366414       0.796930        1.733543          0.0   
min         0.000000       1.000000        0.000000          

In [153]:
# Step 1: Clip the values to the range [15, 24]
HIV_df['age_at_first_sexual_encounter'] = HIV_df['age_at_first_sexual_encounter'].clip(lower=15, upper=24)

# Optional: Check the distribution of the clipped data
print(HIV_df['age_at_first_sexual_encounter'].describe())


count    226256.000000
mean         16.937252
std           1.816676
min          15.000000
25%          15.000000
50%          17.000000
75%          18.000000
max          24.000000
Name: age_at_first_sexual_encounter, dtype: float64


In [154]:
#confirm duplicates and drop them
print(HIV_df.duplicated().sum())
HIV_df = HIV_df.drop_duplicates()

108


In [155]:
missing_values=HIV_df.isna().sum().sort_values(ascending=False)
total_rows=HIV_df.shape[0]
missing_values_percentage=((missing_values/total_rows)*100).round(1)
missing_values_percentage

head_of_household_other              98.7
dreams_program_other                 98.6
exit_reason_other                    94.4
dreams_program                       89.8
used_condom_with_last_partner        53.4
age_of_last_partner                  53.4
age_at_first_sexual_encounter        50.4
has_sexual_partner                   50.3
current_school_level                 44.1
no_of_days_missed_food_in_4wks       43.7
current_income_source                28.4
exit_reason                          15.3
exit_age                             15.1
parents_alive_status                 14.3
age_of_household_head                 4.6
father_alive                          2.9
mother_alive                          2.8
head_of_household                     2.1
county                                2.0
no_of_children                        1.7
ever_had_sex                          1.7
no_of_males                           1.7
no_of_adults                          1.6
no_of_females                     

In [156]:
#Drop columns with % null values above 80%
above_threshold = missing_values_percentage[missing_values_percentage > 80.0]
#print(missing_values_percentage)
print("\nColumns with missing values above 80% threshold:")
print(above_threshold)



Columns with missing values above 80% threshold:
head_of_household_other    98.7
dreams_program_other       98.6
exit_reason_other          94.4
dreams_program             89.8
dtype: float64


In [157]:
# Drop the columns with more than 80% missing values
HIV_df= HIV_df.drop(columns=above_threshold.index)
# Define the percentage range for selecting columns (40% to 80% in this case)
min_percentage = 28.0
max_percentage = 80.0

# Filter columns that have missing values within the percentage range
columns_in_range = missing_values_percentage[(missing_values_percentage >= min_percentage) & (missing_values_percentage <= max_percentage)].index

# Replace missing values with "Unknown" in the selected columns
HIV_df[columns_in_range] = HIV_df[columns_in_range].fillna('Unknown')

# Print the DataFrame after replacement (optional)
print("\nDataFrame after replacing missing values with 'Unknown' in selected columns:")
print(HIV_df[columns_in_range].head())


DataFrame after replacing missing values with 'Unknown' in selected columns:
  used_condom_with_last_partner age_of_last_partner  \
0                       Unknown             Unknown   
1                       Unknown             Unknown   
2                       Unknown             Unknown   
3                       Unknown             Unknown   
4                       Unknown             Unknown   

  age_at_first_sexual_encounter has_sexual_partner current_school_level  \
0                       Unknown            Unknown      Secondary Level   
1                       Unknown            Unknown      Secondary Level   
2                       Unknown            Unknown      Secondary Level   
3                       Unknown            Unknown        Primary Level   
4                       Unknown            Unknown      Secondary Level   

  no_of_days_missed_food_in_4wks current_income_source  
0                        Unknown               Unknown  
1                        U

In [158]:
#Dealing with other nul values where % is less than 28%

# Define the range for missing values percentage
min_percentage = 0.1
max_percentage = 28.0

# Select columns based on missing value percentage within the range
columns_in_range = missing_values_percentage[(missing_values_percentage >= min_percentage) & (missing_values_percentage <= max_percentage)].index
# Step 4: Define a function to replace missing values with "Unknown"
def replace_missing_with_unknown(df, columns):
    df[columns] = df[columns].fillna('Unknown')
    return df

# Step 5: Apply the function to replace missing values in the selected columns
HIV_df = replace_missing_with_unknown(HIV_df, columns_in_range)

# Optional: Check the DataFrame after replacement
print("\nDataFrame after replacing missing values with 'Unknown' in selected columns:")
print(HIV_df[columns_in_range].head())



DataFrame after replacing missing values with 'Unknown' in selected columns:
  exit_reason exit_age parents_alive_status age_of_household_head  \
0     Unknown  Unknown                 Both                  58.0   
1     Unknown  Unknown                 Both                  43.0   
2     Unknown  Unknown                  One                  41.0   
3     Unknown  Unknown                  One                  45.0   
4     Unknown  Unknown                 Both                  42.0   

  father_alive mother_alive head_of_household   county no_of_children  \
0          Yes          Yes            Mother  Nairobi            4.0   
1          Yes          Yes            Father  Nairobi            4.0   
2           No          Yes            Mother  Nairobi            3.0   
3           No          Yes            Mother  Nairobi            4.0   
4          Yes          Yes            Father  Nairobi            2.0   

  ever_had_sex no_of_males no_of_adults no_of_females  \
0          

In [159]:
HIV_df.describe(include=object)

,date_of_birth,date_of_enrollment,county,head_of_household,age_of_household_head,father_alive,mother_alive,are_both_parents_alive,is_one_parent_alive,parents_alive_status,...,age_at_first_sexual_encounter,has_sexual_partner,age_of_last_partner,used_condom_with_last_partner,exit_status,exit_reason,intervention_name,intervention_date,result,exit_age
count,455699,455698,455699,455699,455699.0,455699,455699,455699,455699,455699,...,455699,455699,455699,455699,455699,455699,455699,455699,455699,455699
unique,7839,2604,8,9,69.0,4,4,2,2,3,...,11,3,4,4,2,8,1,2826,2,21
top,1/1/2000,12/9/2020,Nairobi,Father,42.0,Yes,Yes,Yes,No,Both,...,Unknown,Unknown,Unknown,Unknown,Yes,Graduation,HTS - HTS (Client),8/18/2018,Negative,Unknown
freq,1010,1794,100394,173071,26163.0,295775,378810,273135,338517,273135,...,229454,229409,243177,243497,381936,328337,455699,4734,454680,68720


In [160]:
HIV_df

,date_of_birth,date_of_enrollment,county,head_of_household,age_of_household_head,father_alive,mother_alive,are_both_parents_alive,is_one_parent_alive,parents_alive_status,...,exit_reason,intervention_name,intervention_date,result,bio_medical,social_protection,behavioral,post_gbv_care,other_interventions,exit_age
0,9/16/2004,2/22/2020,Nairobi,Mother,58.0,Yes,Yes,Yes,No,Both,...,Unknown,HTS - HTS (Client),8/8/2020,Negative,1,1,1,0,0,Unknown
1,8/2/2004,12/18/2019,Nairobi,Father,43.0,Yes,Yes,Yes,No,Both,...,Unknown,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,Unknown
2,10/20/2005,3/7/2020,Nairobi,Mother,41.0,No,Yes,No,Yes,One,...,Unknown,HTS - HTS (Client),8/12/2020,Negative,1,1,1,0,0,Unknown
3,1/18/2006,3/3/2020,Nairobi,Mother,45.0,No,Yes,No,Yes,One,...,Unknown,HTS - HTS (Client),8/12/2020,Negative,1,1,1,0,0,Unknown
4,3/22/2004,12/18/2019,Nairobi,Father,42.0,Yes,Yes,Yes,No,Both,...,Unknown,HTS - HTS (Client),4/27/2020,Negative,1,1,1,0,0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455802,3/15/2002,3/11/2021,Kiambu,Sibling,19.0,Yes,Yes,Yes,No,Both,...,Lost to follow-up,HTS - HTS (Client),3/11/2021,Negative,1,1,1,0,0,21.0
455803,12/13/1999,4/13/2021,Kiambu,Husband/Partner,21.0,Yes,Yes,Yes,No,Both,...,Graduation,HTS - HTS (Client),8/22/2022,Negative,1,1,1,0,0,22.0
455804,1/5/2007,4/21/2021,Kiambu,Grandparents,54.0,Yes,Yes,Yes,No,Both,...,Graduation,HTS - HTS (Client),3/18/2022,Negative,1,1,1,0,0,16.0
455805,10/12/1997,12/2/2020,Kiambu,Grandparents,71.0,Don't Know,No,No,No,Unknown,...,Graduation,HTS - HTS (Client),12/2/2020,Negative,1,1,1,0,0,25.0
